In [1]:
#Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import keras
from keras.applications.vgg19 import VGG19

In [ ]:
image_size = [224,224]
WIDTH = 151
HEIGHT = 136
BATCH_SIZE = 32
train_path = '/content/drive/MyDrive/March_24/Dataset/Train'
test_path = '/content/drive/MyDrive/March_24/Dataset/Test'

In [ ]:
vgg19 = VGG19(input_shape=image_size+[3],weights="imagenet",include_top=False)

for layers in vgg19.layers:
    layers.trainable=False

In [ ]:
from keras.layers import Dense, Flatten
from keras.models import Sequential, Model
x = Flatten()(vgg19.output)
prediction = Dense(2, activation='softmax')(x)
model = Model(inputs=vgg19.input, outputs=prediction)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [ ]:
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [ ]:
import tensorflow as tf
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
            directory=train_path,
            shuffle=True,
            image_size=(WIDTH,HEIGHT),
            batch_size = BATCH_SIZE,
)

Found 416 files belonging to 2 classes.


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/March_24/Dataset/Train',
                                                target_size = (224,224),
                                                batch_size=32,
                                                class_mode='categorical')

Found 416 images belonging to 2 classes.


In [ ]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/March_24/Dataset/Test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 134 images belonging to 2 classes.


In [ ]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/50
13/13 [==============================] - 416s 32s/step - loss: 1.5811 - accuracy: 0.5673 - val_loss: 0.8391 - val_accuracy: 0.6791
Epoch 2/50
13/13 [==============================] - 387s 30s/step - loss: 0.6827 - accuracy: 0.6538 - val_loss: 1.0230 - val_accuracy: 0.4925
Epoch 3/50
13/13 [==============================] - 381s 30s/step - loss: 0.5569 - accuracy: 0.7115 - val_loss: 0.5305 - val_accuracy: 0.7090
Epoch 4/50
13/13 [==============================] - 380s 30s/step - loss: 0.3849 - accuracy: 0.8197 - val_loss: 0.4601 - val_accuracy: 0.7090
Epoch 5/50
13/13 [==============================] - 430s 34s/step - loss: 0.3156 - accuracy: 0.8750 - val_loss: 0.4223 - val_accuracy: 0.7910
Epoch 6/50
13/13 [==============================] - 375s 29s/step - loss: 0.2733 - accuracy: 0.9135 - val_loss: 0.3939 - val_accuracy: 0.8209
Epoch 7/50
13/13 [==============================] - 381s 30s/step - loss: 0.2671 - accuracy: 0.9111 - val_loss: 0.4118 - val_accuracy: 0.8060
Epoch 

In [ ]:
import matplotlib.pyplot as plt
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
model.save('malaria_model.h5')

In [ ]:
classnames = train_dataset.class_names
classnames
plt.figure(figsize = (15,15))
for image, label in train_dataset.take(1):
    for i in range(12):
        plt.subplot(3, 4, i+1)
        plt.imshow(image[i].numpy().astype('uint8'))
        plt.title(classnames[label[i]])
        plt.axis('off')